### Load relevant packages and sign into Spotify instance

In [15]:
import os, sys, json, requests, urllib.parse
import pandas as pd

import spotipy
import spotipy.util as util

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [3]:
# set API keys
apikeys = json.load(open("data/api-keys.json"))
os.environ["SPOTIPY_CLIENT_ID"]     = apikeys["spotipy-client-id"]
os.environ["SPOTIPY_CLIENT_SECRET"] = apikeys["spotipy-client-secret"]
os.environ["SPOTIPY_REDIRECT_URI"]  = apikeys["redirect-url"]

# set my user_id
user_id = '129874447'

In [4]:
# connect to spotify
token = util.prompt_for_user_token(user_id, \
                                   scope = 'user-library-read, playlist-modify-public, playlist-modify-private')
sp = spotipy.Spotify(auth = token)

### Define helper functions for interfacing with Spotify

In [25]:
### function to load user's saved tracks
def pull_saved_tracks(limit = 50, offset = 0):
    saved_tracks = [ ]
    
    # pull in list of tracks to determine length
    saved_tracks_obj = sp.current_user_saved_tracks(limit = limit, offset = offset)
    num_saved_tracks = saved_tracks_obj['total']
    
    # loop through to pull in all saved tracked
    while (offset < num_saved_tracks):
        saved_tracks_obj = sp.current_user_saved_tracks(limit = limit, offset = offset)
        
        # add track information to running list
        for track_obj in saved_tracks_obj['items']:
            saved_tracks.append({
                'name': track_obj['track']['name'],
                'artists': ', '.join([artist['name'] for artist in track_obj['track']['artists']]),
                'track_id': track_obj['track']['id']
            })
            
        offset += limit
        
    return saved_tracks

### function to load tracks from a specified playlist
def pull_playlist_tracks(user_id, playlist_id, limit = 100, offset = 0):
    playlist_tracks = [ ]
    
    # pull in playlist to determine length
    playlist_obj = sp.user_playlist_tracks(user = user_id, playlist_id = playlist_id, \
                                           limit = limit, offset = offset)
    num_playlist_tracks = playlist_obj['total']
    
    # loop through to pull in all playlist tracks
    while (offset < num_playlist_tracks):
        playlist_obj = sp.user_playlist_tracks(user = user_id, playlist_id = playlist_id, \
                                               limit = limit, offset = offset)

        # add track information to running list
        for track_obj in playlist_obj['items']:
            playlist_tracks.append({
                'name': track_obj['track']['name'],
                'artists': ', '.join([artist['name'] for artist in track_obj['track']['artists']]),
                'track_id': track_obj['track']['id']
            })
            
        offset += limit
        
    return playlist_tracks

### function to load spotify audio features when given a list of track ids
def pull_audio_features(track_ids):
    saved_tracks_audiofeat = [ ]
    
    # iterate through track_ids in groups of 50
    for ix in range(0,len(track_ids),50):
        audio_feats = sp.audio_features(track_ids[ix:ix+50])
        saved_tracks_audiofeat += audio_feats
        
    return saved_tracks_audiofeat

### function to  pull in all user playlists (names and ids)
def pull_all_user_playlists(playlist_limit = 50, playlist_offset = 0):
    playlists_obj = sp.user_playlists(user_id, limit = playlist_limit, offset = playlist_offset)
    num_playlists = playlists_obj['total']

    all_playlists = [{'name': playlist['name'], 'id': playlist['id']} for playlist in playlists_obj['items']]
    playlist_offset += playlist_limit

    while (playlist_offset < num_playlists):
        playlists_obj = sp.user_playlists(user_id, limit = playlist_limit, offset = playlist_offset)
        all_playlists += [{'name': playlist['name'], 'id': playlist['id']} for playlist in playlists_obj['items']]
        playlist_offset += playlist_limit
        
    return(all_playlists)

### Try clustering on pre-made Spotify playlists

In [6]:
# pull tracks for "ambient chill" playlist
testA_tracks    = pull_playlist_tracks(user_id = 'spotify', playlist_id = '37i9dQZF1DX3Ogo9pFvBkY')
testA_tracks_df = pd.DataFrame(testA_tracks)
testA_tracks_df['playlist'] = "ambient chill"

# pull tracks for "hardstyle hits" playlist
testB_tracks    = pull_playlist_tracks(user_id = 'spotify', playlist_id = '37i9dQZF1DX0pH2SQMRXnC')
testB_tracks_df = pd.DataFrame(testB_tracks)
testB_tracks_df['playlist'] = "hardstyle hits"

# stack all tracks together
testAB_tracks_df = testA_tracks_df.append(testB_tracks_df)
testAB_tracks_df.head()

,artists,name,track_id,playlist
0,August Wilhelmsson,Now Is The Time To Leave,6wxi6j0tpjGJGczGrLeDYD,ambient chill
1,Yuki Sakura,Stillness Speaks,3fzTpBMSQKjo9sW0T8Tw2O,ambient chill
2,Eleanor Arroway,Trancendent Sleep,1NY2tT1A7uZwchez1vBShQ,ambient chill
3,chillchild,Gratitude,2ek6LLrUwT6p2zQCCvFmAW,ambient chill
4,Primer Dia,Astral Therapy,17Wjmh3nZzAywevzFHqJnx,ambient chill


In [7]:
# get audio features for stacked set of songs
_testAB_audiofeat    = pull_audio_features(track_ids = list(testAB_tracks_df['track_id']))
_testAB_audiofeat_df = pd.DataFrame(_testAB_audiofeat).drop(['analysis_url', 'track_href', 'type', 'uri'], axis = 1)

_testAB_audiofeat_df.head()

,acousticness,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,0.988,0.200,128000,0.04490,6wxi6j0tpjGJGczGrLeDYD,0.915,0,0.1020,-20.741,0,0.0428,72.278,5,0.0480
1,0.993,0.419,167091,0.00181,3fzTpBMSQKjo9sW0T8Tw2O,0.927,3,0.1380,-33.652,1,0.0408,69.862,4,0.1460
2,0.981,0.353,257463,0.09930,1NY2tT1A7uZwchez1vBShQ,0.928,5,0.1210,-25.317,1,0.0372,138.135,3,0.1230
3,0.969,0.231,159706,0.21700,2ek6LLrUwT6p2zQCCvFmAW,0.968,9,0.2750,-21.127,0,0.0488,112.062,4,0.0405
4,0.949,0.239,137172,0.04720,17Wjmh3nZzAywevzFHqJnx,0.935,4,0.0802,-24.228,0,0.0562,69.666,3,0.0389


In [8]:
# normalize audio features before merging/clustering
testAB_audiofeat_scaler = StandardScaler()

testAB_audiofeat    = testAB_audiofeat_scaler.fit_transform(_testAB_audiofeat_df.drop(['id'], axis = 1))
testAB_audiofeat_df = pd.DataFrame(testAB_audiofeat, columns = _testAB_audiofeat_df.drop('id', axis = 1).columns)
testAB_audiofeat_df['id'] = _testAB_audiofeat_df['id']

testAB_audiofeat_df.head()

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,id
0,0.965232,-0.739829,-0.994808,-0.965980,0.810399,-1.175325,-0.523828,-0.612784,-1.321022,-0.301368,-1.226364,1.675060,-0.759296,6wxi6j0tpjGJGczGrLeDYD
1,0.977137,0.533462,-0.419151,-1.082851,0.839429,-0.368729,-0.265242,-2.025394,0.756990,-0.338052,-1.296990,0.295599,0.040835,3fzTpBMSQKjo9sW0T8Tw2O
2,0.948564,0.149730,0.911675,-0.818434,0.841849,0.169001,-0.387352,-1.113450,0.756990,-0.404082,0.698803,-1.083862,-0.146951,1NY2tT1A7uZwchez1vBShQ
3,0.919990,-0.559592,-0.527903,-0.499201,0.938617,1.244461,0.718826,-0.655016,-1.321022,-0.191318,-0.063377,0.295599,-0.820531,2ek6LLrUwT6p2zQCCvFmAW
4,0.872368,-0.513079,-0.859741,-0.959742,0.858783,-0.099864,-0.680417,-0.994301,-1.321022,-0.055589,-1.302720,-1.083862,-0.833594,17Wjmh3nZzAywevzFHqJnx


In [47]:
# merge track info with normalized audio features
testAB_tracks_plus_df = testAB_tracks_df.merge(testAB_audiofeat_df, how = 'left', \
                                               left_on = 'track_id', right_on = 'id')
testAB_tracks_plus_df.head()

,artists,name,track_id,playlist,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,id
0,August Wilhelmsson,Now Is The Time To Leave,6wxi6j0tpjGJGczGrLeDYD,ambient chill,0.965232,-0.739829,-0.994808,-0.965980,0.810399,-1.175325,-0.523828,-0.612784,-1.321022,-0.301368,-1.226364,1.675060,-0.759296,6wxi6j0tpjGJGczGrLeDYD
1,Yuki Sakura,Stillness Speaks,3fzTpBMSQKjo9sW0T8Tw2O,ambient chill,0.977137,0.533462,-0.419151,-1.082851,0.839429,-0.368729,-0.265242,-2.025394,0.756990,-0.338052,-1.296990,0.295599,0.040835,3fzTpBMSQKjo9sW0T8Tw2O
2,Eleanor Arroway,Trancendent Sleep,1NY2tT1A7uZwchez1vBShQ,ambient chill,0.948564,0.149730,0.911675,-0.818434,0.841849,0.169001,-0.387352,-1.113450,0.756990,-0.404082,0.698803,-1.083862,-0.146951,1NY2tT1A7uZwchez1vBShQ
3,chillchild,Gratitude,2ek6LLrUwT6p2zQCCvFmAW,ambient chill,0.919990,-0.559592,-0.527903,-0.499201,0.938617,1.244461,0.718826,-0.655016,-1.321022,-0.191318,-0.063377,0.295599,-0.820531,2ek6LLrUwT6p2zQCCvFmAW
4,Primer Dia,Astral Therapy,17Wjmh3nZzAywevzFHqJnx,ambient chill,0.872368,-0.513079,-0.859741,-0.959742,0.858783,-0.099864,-0.680417,-0.994301,-1.321022,-0.055589,-1.302720,-1.083862,-0.833594,17Wjmh3nZzAywevzFHqJnx


In [10]:
# try clustering full stack of songs into two playlists
num_clusters = 2
kmeans = KMeans(n_clusters = num_clusters).fit(testAB_tracks_plus_df.drop(['track_id', 'id', 'name', 'artists', \
                                                                           'playlist'], axis = 1))
testAB_tracks_plus_df['cluster'] = pd.Series(kmeans.labels_) + 1

# see if successful
testAB_tracks_plus_df[['track_id', 'playlist', 'cluster']].groupby(['playlist', 'cluster']).agg('count')

,,track_id
playlist,cluster,
ambient chill,1,90
hardstyle hits,2,50


Since the playlists were grouped into mutually exclusive clusters, we can see that this approach works, at least on quite different sounding sets of songs.

As a secondary experiment, before trying it on my own library, I want to see how it performs on more similar playlists.

But first, going to throw this code into some functions for later use.

In [11]:
### function to create "tracks plus" df (including normalized audio features) when given a tracks df
def build_tracks_plus_df(tracks_df, normalize = True):
    # pull raw audio features
    _audiofeat    = pull_audio_features(track_ids = list(tracks_df['track_id']))
    _audiofeat_df = pd.DataFrame(_audiofeat).drop(['analysis_url', 'track_href', 'type', 'uri'], axis = 1)
    
    # scale audio features (if desired)
    if normalize:
        scaler = StandardScaler()
        audiofeat    = scaler.fit_transform(_audiofeat_df.drop(['id'], axis = 1))
        audiofeat_df = pd.DataFrame(audiofeat, columns = _audiofeat_df.drop('id', axis = 1).columns)
        audiofeat_df['id'] = _audiofeat_df['id']
    else:
        audiofeat_df = _audiofeat_df
    
    # merge audio features with tracks_df
    tracks_plus_df = tracks_df.merge(audiofeat_df, how = 'left', left_on = 'track_id', right_on = 'id')
    return(tracks_plus_df)

### function to cluster tracks based on normalized audio features
def cluster_tracks_plus_df(tracks_plus_df, num_clusters, drop_vars = None):
    kmeans = KMeans(n_clusters = num_clusters).fit(tracks_plus_df.drop(['track_id', 'id', 'name', 'artists'] + \
                                                                       (drop_vars if drop_vars != None else []), \
                                                                       axis = 1))
    tracks_plus_df['cluster'] = pd.Series(kmeans.labels_) + 1
    return(tracks_plus_df)

In [43]:
# pull tracks for "lo-fi indie" playlist
testC_tracks    = pull_playlist_tracks(user_id = 'spotify', playlist_id = '37i9dQZF1DX0CIO5EOSHeD')
testC_tracks_df = pd.DataFrame(testC_tracks)
testC_tracks_df['playlist'] = "lo-fi indie"

# pull tracks for "dreampop" playlist
testD_tracks    = pull_playlist_tracks(user_id = 'spotify', playlist_id = '37i9dQZF1DX6uhsAfngvaD')
testD_tracks_df = pd.DataFrame(testD_tracks)
testD_tracks_df['playlist'] = "dreampop"

# pull tracks for "bedroom pop" playlist
testE_tracks    = pull_playlist_tracks(user_id = 'spotify', playlist_id = '37i9dQZF1DXcxvFzl58uP7')
testE_tracks_df = pd.DataFrame(testE_tracks)
testE_tracks_df['playlist'] = "bedroom pop"

# stack all tracks together
testCDE_tracks_df = testC_tracks_df.append(testD_tracks_df).append(testE_tracks_df)
testCDE_tracks_df.head()

# build plus df and cluster
testCDE_tracks_plus_df = cluster_tracks_plus_df(build_tracks_plus_df(testCDE_tracks_df), 3, drop_vars = ['playlist'])
testCDE_tracks_plus_df[['track_id', 'playlist', 'cluster']].groupby(['playlist', 'cluster']).agg('count')

track_id
playlist    cluster          
bedroom pop 1              28
            2              94
            3              12
dreampop    1              52
            2              10
            3               2
lo-fi indie 1              33
            2              20
            3               1

The results aren't as clean on this try, but that makes sense because these are similar sounding playlists. On my own listening, it wasn't entirely obvious how one song would fall into one or another, so it would be a lot to expect the clustering algorithm to do it.

Nonetheless, the goal of this experiment is not to make perfectly partitioned playlists purely based on Spotify's own genres, but instead to create playlists that have similar vibes, hopefully grouping together songs that are not entirely obvious at first listen.

### Pull track and audiofeature information for all of my saved tracks

In [13]:
# pull in list of saved songs
saved_tracks_df      = pd.DataFrame(pull_saved_tracks())
saved_tracks_plus_df = build_tracks_plus_df(saved_tracks_df, )

In [34]:
saved_tracks_clustered1_df = cluster_tracks_plus_df(saved_tracks_plus_df, 200)
saved_tracks_clustered1_df[saved_tracks_clustered1_df['cluster'] == 1]

,artists,name,track_id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,id,cluster
237,Carla Thomas,B-A-B-Y,35w6fQ0Wnuv7aIryRx7gTq,0.275808,1.425729,-0.867540,-1.250747,-0.409308,0.880152,-0.668430,-1.020603,0.557102,-0.173422,-0.220099,0.269713,1.035343,35w6fQ0Wnuv7aIryRx7gTq,1
274,Wild Child,Break You Down,7xMtPZMknqTa8Y9cSJV6y6,-0.558158,1.220328,-0.493045,-0.309459,-0.392442,0.045020,0.590713,-0.616231,0.557102,-0.201094,-1.473143,0.269713,1.555997,7xMtPZMknqTa8Y9cSJV6y6,1
369,"Rhye, Roosevelt",Summer Days - Roosevelt Remix,41Voit0h2hfPclo2RRaX4c,0.067316,1.932384,-0.395276,-0.460420,1.188307,0.601775,-0.716858,0.058694,0.557102,-0.431699,-0.340657,0.269713,1.807647,41Voit0h2hfPclo2RRaX4c,1
403,"Otis Redding, Carla Thomas",It Takes Two,65bUBWQHR0rQjcX4jitkZA,0.717319,0.919074,-0.758682,-0.003096,-0.408951,1.436906,0.007086,-1.517191,0.557102,-0.101165,1.060577,0.269713,2.345657,65bUBWQHR0rQjcX4jitkZA,1
405,"Otis Redding, Carla Thomas",Bring It On Home To Me,35uBp19uiQXgztg8NRQOkT,0.285006,0.857454,-1.241041,-1.419468,-0.409308,0.880152,-0.464781,-2.470619,0.557102,-0.411714,-0.821490,-2.140549,1.963844,35uBp19uiQXgztg8NRQOkT,1
413,Otis Redding,Scratch My Back,6HSYhD3f1zhfuuMfOOnlC4,0.576280,2.172018,-1.081902,-1.135306,-0.113211,0.323397,-0.975765,-1.463664,0.557102,-0.158048,-0.830722,0.269713,2.037603,6HSYhD3f1zhfuuMfOOnlC4,1
434,Otis Redding,Ole Man Trouble,6IxGVlmZlYIp0RUwKnR6oc,0.560950,0.850607,-1.179478,-1.166386,-0.407931,1.158529,-0.166760,-1.105664,0.557102,-0.453222,-1.511874,0.269713,0.909518,6IxGVlmZlYIp0RUwKnR6oc,1
731,Wilson Pickett,Hey Jude,1MMp1H2Kib2BCDtdL5nL63,-0.588819,0.200172,0.182492,-0.877784,-0.406985,0.323397,-0.092254,-1.909639,0.557102,-0.394802,-1.287057,0.269713,1.013649,1MMp1H2Kib2BCDtdL5nL63,1
847,Sam & Dave,"Hold On, I'm Comin'",6PgVDY8GTkxF3GmhVGPzoB,-0.435516,1.857070,-1.183635,-1.086466,-0.231275,0.323397,-0.551704,-1.564889,0.557102,-0.354831,-0.445716,0.269713,2.089668,6PgVDY8GTkxF3GmhVGPzoB,1
903,Al Green,Tired of Being Alone,5vjmFhes2UVEo5aGQ27q7l,0.082646,1.624283,-0.937015,-0.855584,-0.360583,0.601775,-0.736105,-0.106394,0.557102,-0.290262,-0.737829,0.269713,0.779354,5vjmFhes2UVEo5aGQ27q7l,1


In [35]:
def save_cluster_tracks_to_playlist(playlist_name, track_ids):
    # get all of the users playlists
    all_playlists = pull_all_user_playlists()
    
    # check if playlist already exists
    if (playlist_name not in [playlist['name'] for playlist in all_playlists]):
        playlist = sp.user_playlist_create(user = user_id, name = playlist_name, public = True)
    else:
        playlist_id = [playlist['id'] for playlist in all_playlists if playlist['name'] == playlist_name][0]
        playlist = sp.user_playlist(user = user_id, playlist_id = playlist_id)

    # remove any existing tracks in playlist
    while (playlist['tracks']['total'] > 0):
        sp.user_playlist_remove_all_occurrences_of_tracks(user_id, playlist['id'], \
                                                          tracks = [track['track']['id'] for track in \
                                                                    playlist['tracks']['items']])
        playlist = sp.user_playlist(user = user_id, playlist_id = playlist_id)

    # add tracks from cluster
    cluster_ix = 50
    sp.user_playlist_add_tracks(user_id, playlist_id = playlist['id'], tracks = track_ids)

In [38]:
save_cluster_tracks_to_playlist("k-means, cluster 1", \
                                list(saved_tracks_clustered1_df[saved_tracks_clustered1_df['cluster'] == 1]['id']))
save_cluster_tracks_to_playlist("k-means, cluster 2", \
                                list(saved_tracks_clustered1_df[saved_tracks_clustered1_df['cluster'] == 2]['id']))
save_cluster_tracks_to_playlist("k-means, cluster 3", \
                                list(saved_tracks_clustered1_df[saved_tracks_clustered1_df['cluster'] == 24]['id']))

In [40]:
saved_tracks_clustered1_df[saved_tracks_clustered1_df['name'] == "Upgrade"]

,artists,name,track_id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,id,cluster
544,Logic,Upgrade,1kUnxg3OKR5itogg7WSX51,-0.740282,0.932767,-0.919601,1.071393,-0.409277,0.601775,-0.353023,0.533553,-1.795003,0.267803,1.525544,0.269713,-0.392119,1kUnxg3OKR5itogg7WSX51,148


In [41]:
saved_tracks_clustered1_df[saved_tracks_clustered1_df['cluster'] == 148]

,artists,name,track_id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,id,cluster
260,Jacob Banks,Chainsmoking,1pfzqOGRJd6DbEIFNE5G1X,0.622271,-1.032231,-0.598355,0.445348,-0.409308,0.323397,-0.390276,1.079428,-1.795003,0.707490,0.598410,0.269713,0.722950,1pfzqOGRJd6DbEIFNE5G1X,148
283,"Justin Timberlake, Alicia Keys",Morning Light,3KnLHwhRhJGuQM822JVSoI,0.134769,0.604127,0.112825,-0.380499,-0.183487,1.158529,0.044339,-0.387281,-1.795003,0.938095,1.189734,0.269713,0.519027,3KnLHwhRhJGuQM822JVSoI,148
517,"GoGo Penguin, J Fogel",Initiate - Stray Remix,35pZwiqulpnv2sHXH4hRTb,0.809300,-0.867911,-0.309085,-0.957704,-0.317481,0.601775,-0.565985,-1.262007,-1.795003,1.337810,1.865953,0.269713,-0.301004,35pZwiqulpnv2sHXH4hRTb,148
537,Logic,Lord Willin',0HHvRJqPIoaCe934zxaM73,-0.640942,0.747907,-0.397443,1.235675,-0.408339,0.323397,-0.816199,0.941370,-1.795003,-0.131913,1.662201,0.269713,0.679562,0HHvRJqPIoaCe934zxaM73,148
544,Logic,Upgrade,1kUnxg3OKR5itogg7WSX51,-0.740282,0.932767,-0.919601,1.071393,-0.409277,0.601775,-0.353023,0.533553,-1.795003,0.267803,1.525544,0.269713,-0.392119,1kUnxg3OKR5itogg7WSX51,148
550,Father John Misty,Total Entertainment Forever,2EDu3Xi4LORS0HRJ7kt0hF,-0.499903,-0.991151,-0.913663,0.964832,-0.409279,0.601775,-0.241265,0.868233,-1.795003,-0.105777,1.252463,0.269713,0.601464,2EDu3Xi4LORS0HRJ7kt0hF,148
583,Kaleo,Way Down We Go,0y1QJc3SJVPKJ1OvFmFqe6,0.738781,-0.292789,-0.322624,-0.344979,-0.407748,1.436906,-0.508243,0.026895,-1.795003,0.907348,1.434617,0.269713,-0.405135,0y1QJc3SJVPKJ1OvFmFqe6,148
713,NEEDTOBREATHE,MONEY & FAME,4jtJZ9mep9Cdh8RGl5vw0g,-0.826131,0.152245,-0.642683,1.368876,-0.409240,0.323397,-0.592062,1.114142,-1.795003,0.632159,1.126539,0.269713,0.102503,4jtJZ9mep9Cdh8RGl5vw0g,148
807,NEEDTOBREATHE,MONEY & FAME,22JGzWpzuBQMbMMxq6egMq,-0.826131,0.152245,-0.642683,1.368876,-0.409240,0.323397,-0.592062,1.114142,-1.795003,0.632159,1.126539,0.269713,0.102503,22JGzWpzuBQMbMMxq6egMq,148
1010,Otis Redding,Shake - Remastered Stereo,6RkyopJ2y0DnoIrq57zrap,-0.423252,0.350799,-1.097935,-0.495940,-0.391974,0.601775,-0.613171,-0.420935,-1.795003,0.172486,1.438250,0.269713,1.586369,6RkyopJ2y0DnoIrq57zrap,148
